# T1098.002 - Additional Email Delegate Permissions
Adversaries may grant additional permission levels to maintain persistent access to an adversary-controlled email account. 

For example, the <code>Add-MailboxPermission</code> [PowerShell](https://attack.mitre.org/techniques/T1059/001) cmdlet, available in on-premises Exchange and in the cloud-based service Office 365, adds permissions to a mailbox.(Citation: Microsoft - Add-MailboxPermission)(Citation: FireEye APT35 2018)(Citation: Crowdstrike Hiding in Plain Sight 2018) In Google Workspace, delegation can be enabled via the Google Admin console and users can delegate accounts via their Gmail settings.(Citation: Gmail Delegation)(Citation: Google Ensuring Your Information is Safe) 

Adversaries may also assign mailbox folder permissions through individual folder permissions or roles. In Office 365 environments, adversaries may assign the Default or Anonymous user permissions or roles to the Top of Information Store (root), Inbox, or other mailbox folders. By assigning one or both user permissions to a folder, the adversary can utilize any other account in the tenant to maintain persistence to the target user’s mail folders.(Citation: Remediation and Hardening Strategies for Microsoft 365 to Defend Against UNC2452)

This may be used in persistent threat incidents as well as BEC (Business Email Compromise) incidents where an adversary can add [Additional Cloud Roles](https://attack.mitre.org/techniques/T1098/003) to the accounts they wish to compromise. This may further enable use of additional techniques for gaining access to systems. For example, compromised business accounts are often used to send messages to other accounts in the network of the target business while creating inbox rules (ex: [Internal Spearphishing](https://attack.mitre.org/techniques/T1534)), so the messages evade spam/phishing detection mechanisms.(Citation: Bienstock, D. - Defending O365 - 2019)

## Atomic Tests

### Atomic Test #1 - EXO - Full access mailbox permission granted to a user
Give a nominated user, full mailbox delegation access of another user.
This can be used by an adversary to maintain persistent access to a target's mailbox in M365.

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement         

```

In [ ]:
Invoke-AtomicTest T1098.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module ExchangeOnlineManagement
$secure_pwd = "o365_password_test" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "o365_user_test@contoso.com", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Add-MailboxPermission -Identity "delegate@contoso.com" -User "operator@contoso.com" -AccessRights FullAccess -InheritanceType All
Disconnect-ExchangeOnline -Confirm:$false
```

In [ ]:
Invoke-AtomicTest T1098.002 -TestNumbers 1

#### Cleanup: 
```powershell
Import-Module ExchangeOnlineManagement
$secure_pwd = "o365_password_test" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "o365_user_test@contoso.com", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Remove-MailboxPermission -Identity "delegate@contoso.com" -User "operator@contoso.com" -AccessRights FullAccess -InheritanceType All -Confirm:$false
Disconnect-ExchangeOnline -Confirm:$false
```

In [ ]:
Invoke-AtomicTest T1098.002 -TestNumbers 1 -Cleanup

## Detection
Monitor for unusual Exchange and Office 365 email account permissions changes that may indicate excessively broad permissions being granted to compromised accounts.

Enable the UpdateFolderPermissions action for all logon types. The mailbox audit log will forward folder permission modification events to the Unified Audit Log. Create rules to alert on ModifyFolderPermissions operations where the Anonymous or Default user is assigned permissions other than None. 

A larger than normal volume of emails sent from an account and similar phishing emails sent from  real accounts within a network may be a sign that an account was compromised and attempts to leverage access with modified email permissions is occurring.